In [1]:
import xarray
import cmocean as cm
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from pathlib import Path
from xmovie import Movie
import matplotlib.path as mpath
import cartopy.feature as cft
import numpy as np
import intake 
import sys  

from pathlib import Path

from os import environ
environ["PYTHONWARNINGS"] = "ignore"

import warnings
import xarray as xr
warnings.filterwarnings("ignore") # Suppress warnings for these docs

In [2]:
import cosima_cookbook as cc
db = 'local_cc_scratch_Ryan_data.db'
session = cc.database.create_session(db)

In [3]:
# %%time
# directory_list=['/g/data/e14/rmh561/access-om2/archive/025deg_jra55_iaf_cycle6_online_mlt']
# cc.database.build_index(directory_list, session)

In [4]:
from dask.distributed import Client

client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 14,Total memory: 510.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42817,Workers: 14
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 510.00 GiB
Comm: tcp://127.0.0.1:40101,Total threads: 1
Dashboard: /proxy/35481/status,Memory: 36.43 GiB
Nanny: tcp://127.0.0.1:33271,


# SST - Create Zarr Stores

In [22]:
data = cc.querying.getvar(
        expt='025deg_jra55_iaf_cycle6_online_mlt',
        variable='sss',
        session=session,
        frequency='1 daily',
        attrs={'cell_methods': 'time: mean'},
    ).sel(time=slice('1982', '2011')).chunk({
    'time': -1,
    'xt_ocean': 84,   
    'yt_ocean': 20   
}).sel(yt_ocean = slice(-71,-62)).sel(xt_ocean = slice(-65,25)).rename({'xt_ocean': 'lon', 'yt_ocean': 'lat'})

In [24]:
data.load()

<xarray.DataArray 'sss' (time: 10957, lat: 84, lon: 360)> Size: 1GB
array([[[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [33.797012, 33.802788, 33.81495 , ..., 33.350456, 33.34831 ,
         33.345478],
        [33.797295, 33.805954, 33.819073, ..., 33.35063 , 33.347782,
         33.34737 ],
        [33.799828, 33.81093 , 33.826397, ..., 33.351967, 33.350716,
         33.34581 ]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
...
        [33.64339 , 33.65579 , 33.675873, ..., 33.9085  , 33.907684,
         33.914177],
        [33.63413 , 33.646454, 33.666676, ..., 33.90446 , 33.888924,
         33.898403],
        [33.63173 , 33.646606, 33.667557, ..., 33.902542, 33.880722,
         33.879738]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [33.639027, 33.652046, 33.671547, ..., 33.894524, 33.89953 ,
         33.89971 ],
        [33.63011 , 33.642635, 33.66568 , ..., 33.862713, 33.873306,
         33.890198],
        [33.62702 , 33.642555, 33.664005, ..., 33.840458, 33.836124,
         33.8537  ]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 3kB -64.88 -64.62 -64.38 -64.12 ... 24.38 24.62 24.88
  * lat      (lat) float64 672B -70.94 -70.83 -70.73 ... -62.25 -62.14 -62.02
  * time     (time) datetime64[ns] 88kB 1982-01-01T12:00:00 ... 2011-12-31T12...
Attributes:
    long_name:      Practical Salinity
    units:          psu
    valid_range:    [-10. 100.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_surface_salinity
    ncfiles:        ['/g/data/e14/rmh561/access-om2/archive/025deg_jra55_iaf_...
    contact:        Ryan Holmes
    email:          ryan.holmes@unsw.edu.au
    created:        2020-11-03
    description:    0.25 degree ACCESS-OM2 global model configuration under i...

In [25]:
#Write SST to a zarr datastore to prevent memory blowing up when doing thresholds
data.to_zarr('/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_zarr/025deg_jra55_iaf_cycle6_online_mlt_zarr_20_to_m60_lat_m71_to_m62.zarr')

In [9]:
ds_input = xr.open_dataset('/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_zarr/025deg_jra55_iaf_cycle6_online_mlt_zarr_20_to_m60_lat_m71_to_m62.zarr')
print(ds_input)

<xarray.Dataset> Size: 1GB
Dimensions:  (lat: 84, lon: 360, time: 10957)
Coordinates:
  * lat      (lat) float64 672B -70.94 -70.83 -70.73 ... -62.25 -62.14 -62.02
  * lon      (lon) float64 3kB -64.88 -64.62 -64.38 -64.12 ... 24.38 24.62 24.88
  * time     (time) datetime64[ns] 88kB 1982-01-01T12:00:00 ... 2011-12-31T12...
Data variables:
    sss      (time, lat, lon) float32 1GB ...


## SST - Create Climatologies

In [6]:
import os
os.chdir('/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/MHW_Toolbox')
import bipolarMhwToolBox as MHW

In [7]:
import xarray as xr
import glob
import os

# Input and output directories
zarr_dir = '/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_zarr/'  
output_dir = '/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_clim/'

os.makedirs(output_dir, exist_ok=True)

# Get all matching Zarr store paths
zarr_stores = glob.glob(os.path.join(zarr_dir, '025deg_jra55_iaf_cycle6_online_mlt_zarr_*.zarr'))

for zarr_path in zarr_stores:
    base_name = os.path.basename(zarr_path).replace('.zarr', '').replace('rechunked_', 'clim_rechunked_')
    print(base_name)
    output_path = os.path.join(output_dir, f'{base_name}.nc')

    if os.path.exists(output_path):
        print(f'Skipping (already exists): {output_path}')
        continue

    print(f'Processing: {zarr_path}')
    
    ds_input = xr.open_zarr(zarr_path).chunk(dict(time=-1))
    print(ds_input)

    ds_input = ds_input.load()
    # Compute climatologies
    Seas = MHW.smoothedClima_mhw(ds_input.sss)

    # Persist result before writing
    Seas.load()
    
    # Write to NetCDF
    print(f'Writing to: {output_path}')
    Seas.to_netcdf(output_path)

025deg_jra55_iaf_cycle6_online_mlt_zarr_20_to_m60_lat_m71_to_m62
Processing: /g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_zarr/025deg_jra55_iaf_cycle6_online_mlt_zarr_20_to_m60_lat_m71_to_m62.zarr
<xarray.Dataset> Size: 1GB
Dimensions:  (lat: 84, lon: 360, time: 10957)
Coordinates:
  * lat      (lat) float64 672B -70.94 -70.83 -70.73 ... -62.25 -62.14 -62.02
  * lon      (lon) float64 3kB -64.88 -64.62 -64.38 -64.12 ... 24.38 24.62 24.88
  * time     (time) datetime64[ns] 88kB 1982-01-01T12:00:00 ... 2011-12-31T12...
Data variables:
    sss      (time, lat, lon) float32 1GB dask.array<chunksize=(10957, 11, 45), meta=np.ndarray>
Writing to: /g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_clim/025deg_jra55_iaf_cycle6_online_mlt_zarr_20_to_m60_lat_m71_to_m62.nc


# SST - Create Thresholds

In [8]:
import xarray as xr
import glob
import os

# Input and output directories
zarr_dir = '/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_zarr'  
output_dir = '/g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_Thresholds90thPercentile'

os.makedirs(output_dir, exist_ok=True)

# Get all matching Zarr store paths
zarr_stores = glob.glob(os.path.join(zarr_dir, '025deg_jra55_iaf_cycle6_online_mlt_zarr_*.zarr'))

for zarr_path in zarr_stores:
    
    # Construct output filename
    base_name = os.path.basename(zarr_path).replace('.zarr', '').replace('rechunked_', 'clim_rechunked_')
    output_path = os.path.join(output_dir, f'{base_name}.nc')

    # Skip if already processed
    if os.path.exists(output_path):
        print(f'Skipping (already exists): {output_path}')
        continue

    print(f'\nProcessing: {zarr_path}')
    
    # Open the Zarr dataset
    ds_input = xr.open_zarr(zarr_path).chunk({'time': -1})
    print(f'Dataset loaded with variables: {list(ds_input.data_vars)}')
    ds_input = ds_input.load()

    # Compute 90th percentile threshold from the function
    Thresh = MHW.smoothedThresh_mhw(ds_input.sss)

    # Persist result before writing
    Thresh = Thresh.load()
    
    # Save to NetCDF
    print(f'Writing to: {output_path}')
    Thresh.to_netcdf(output_path)


Processing: /g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_zarr/025deg_jra55_iaf_cycle6_online_mlt_zarr_20_to_m60_lat_m71_to_m62.zarr
Dataset loaded with variables: ['sss']
Writing to: /g/data/e14/ts7341/Analysis_Using_Daily_JRA/JRA_SST_Processed/sss_Thresholds90thPercentile/025deg_jra55_iaf_cycle6_online_mlt_zarr_20_to_m60_lat_m71_to_m62.nc
